# Daten Aggregation und Datenbereinigung
Dieses Notebook wird anhand von Escherichia coli die Datenbeschaffung und Bereinigung aufzeigen.
## Download 
Als erstes müssen die Genome Sequenzen auf der Seite des [ncbi](https://www.ncbi.nlm.nih.gov/datasets/taxonomy/562/ ) heruntergeladen werden.
![image.png](images\DownloadFasta.png)

## FASTA Files
Man erhält eine Fasta Datei mit allen Genomen von E.Coli . Dort beginnt jeder Eintrag mit einem > gefolgt von der ID und einer Beschreibung anschließend befinden sich die Daten. In unserem Fall sind die jeweiligen Codons aufgeführt.

## Clean and Pickle Data
Um fehlerhafte Daten auszusortieren und die so gewonnen Daten zu speichern, wurde im [clean_and_pickle Skript](../scripts/clean_and_pickle.py) die Bibliothek Biopython und Pandas genutzt.
Dort wird überprüft ob eine Sequenz:
- mit dem Startcodon beginnt
- die Anzahl der Buchstaben ohne Rest durch drei teilbar ist
- ob es Stopcodons innerhalb der Sequenz gibt
- es nur Valide Übersetzung gibt
- keine doppelten Daten existieren
  
Anschließend wird ein Report wie viele Sequenzen aus welchen Gründen nicht gespeichert wurden, sowie die eigentlichen Sequenzen mit deren Übersetzung gespeichert. 
Für neue Organismen müssen die Speicherorte der Fasta Datei und der gepickelten Daten angepasst werden.  

In [6]:
import ipywidgets as widgets
choosenOrganism = widgets.Dropdown(options=['E.Coli','Drosophila.Melanogaster','Homo.Sapiens','saccharomyces.cerevisiae'],value='E.Coli',Description='Organism',disable=False)
display(choosenOrganism)

Dropdown(options=('E.Coli', 'Drosophila.Melanogaster', 'Homo.Sapiens', 'saccharomyces.cerevisiae'), value='E.C…

In [8]:
import sys
sys.path.append('../scripts')

import clean_and_pickle as cap
cap.clean_and_pickle(organism=choosenOrganism.value)


{'modThree': 0, 'stopCodon': 6, 'doubleData': 33, 'noStartCodon': 0}


Die Daten werden anschließend mit dem [data_splitter Skript](../scripts/data_splittere.py)  in Trainings(80%), Test(10%) und Validierungsdaten (10%) aufgeteilt. Die entsprechenden Datensätze werden ebenfalls als pickel im Data Ordner gespeichert.

In [3]:
import sys
sys.path.append('../scripts')
import data_splitter as ds
ds.split_n_pickle(organism=choosenOrganism.value)

E.Coli                    total:   3856 | train:   3085 | test:    386 | valid:    385


## Codon Häufigkeit

Nun wird, die relative Häufigkeit wie oft eine Aminosäure von einem Codon übersetzt wird berechnet und gespeichert. Dazu wird das Skript [Usage bias and pickle](../scripts/usage_bias_and_pickle.py)

In [4]:
import sys
sys.path.append('../scripts')

import usage_bias_and_pickle as ubap
ubap.main('../data', organism=choosenOrganism.value)

Calculating usage bias for: E.Coli
usage bias will be saved in: ../data\E.Coli


## Struktus als weiteres Feature
Nachdem wir die primär Struktur, also die reine Aneinanderreihung der Aminosäuren beschafft haben, war eine weiteres Ziel die Struktur der Proteine zu verwenden, um die richtigen Codons vorherzusagen.
Dabei gab es zwei unterschiedliche Ansätze. 
### Sekundär Struktur
Bei dem ersten Ansatz ging es um die sekundär Struktur des Proteins, also ob in welcher Struktur befindet sich eine Aminosäure, innerhalb des Proteins. Ein auslesen wäre mit dem [DSSP Modul](https://biopython.org/docs/1.76/api/Bio.PDB.DSSP.html) von Biopython denkbar gewesen. Jedoch können die Daten nicht mit den IDs, welche wir aus den FASTA Dateien gewinnen konnten, gesucht werden. Um einen Einblick in vorhandene DSSP Daten zu gewinnen, lohnt sich ein Blick in diese [DSSP Datenbank](https://mrs.cmbi.umcn.nl/search?db=dssp&q=562&count=3).
### Quartäre Struktur
Mit der selben ID lassen sich die Position im Raum der einzelnen Atome der Aminosäuren, in zusammenspiel mit anderen Proteinen ausgeben. Das Package [PDBParser](https://biopython.org/docs/1.76/api/Bio.PDB.PDBParser.html) wurde im Skript [Secondary Structure](../scripts/secondary_structur.py) dafür verwendet. Die [RCSB Protein Data Bank](https://www.rcsb.org/) wurde zum finden der IDs für E.Coli genutzt.

Da die zeitaufwendigen Versuchen Codons und ihre Aminosäure einer Struktur zuzuordnen nicht klappten bzw. nur eine geringe Anzahl an Matches aufwies, wurden beide Ansätze zugunsten anderer Aufgaben innerhalb des Projektes nicht weiterverfolgt.